In [3]:
from fastai.text.all import *
path = Path("data")
files = get_text_files(path, folders = ['train', 'test'])

In [10]:
txts = L(["Fourscore and seven years ago, our fathers brought forth upon this continent a new nation, conceived in liberty, and dedicated to the pursuit of happiness.",
          "No, you don't understand!"])
txt = txts[0]

In [21]:
tkn = Tokenizer(WordTokenizer())
toks = tkn(txt)
print(coll_repr(toks, 30))

(#31) ['xxbos','xxmaj','fourscore','and','seven','years','ago',',','our','fathers','brought','forth','upon','this','continent','a','new','nation',',','conceived','in','liberty',',','and','dedicated','to','the','pursuit','of','happiness'...]


In [15]:
toks_all = txts.map(tkn)
toks_all

(#2) [['xxbos', 'xxmaj', 'fourscore', 'and', 'seven', 'years', 'ago', ',', 'our', 'fathers', 'brought', 'forth', 'upon', 'this', 'continent', 'a', 'new', 'nation', ',', 'conceived', 'in', 'liberty', ',', 'and', 'dedicated', 'to', 'the', 'pursuit', 'of', 'happiness', '.'],['xxbos', 'xxmaj', 'no', ',', 'you', 'do', "n't", 'understand', '!']]

In [19]:
num = Numericalize(min_freq=1)
num.setup(toks_all)
print(coll_repr(num.vocab,20))

(#48) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj',',','and','fourscore','seven','years','ago','our','fathers','brought','forth','upon'...]


In [22]:
nums = num(toks)[:20]; nums

TensorText([ 2,  8, 11, 10, 12, 13, 14,  9, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
         9, 25])

In [23]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj fourscore and seven years ago , our fathers brought forth upon this continent a new nation , conceived'

In [24]:
nums_all = toks_all.map(num)

In [27]:
dl = LMDataLoader(nums_all)

Creating the language model:

In [98]:
get_files = partial(get_text_files, folders=['train', 'test'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_files, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [99]:
dls_lm.show_batch(max_n=2)

/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The ser

,text,text_
0,"xxbos xxmaj xxunk xxunk is a moth of the xxmaj noctuidae family . xxbos xxmaj while these buildings are notable as xxunk of the history of xxunk in xxmaj hong xxmaj kong and of the public xxunk policy of the xxmaj government of xxmaj hong xxmaj kong , they xxunk only a xxunk of the industrial buildings of the territory : there were about xxunk industrial buildings in xxmaj hong xxmaj kong in 2003 . xxbos xxmaj good xxunk .","xxmaj xxunk xxunk is a moth of the xxmaj noctuidae family . xxbos xxmaj while these buildings are notable as xxunk of the history of xxunk in xxmaj hong xxmaj kong and of the public xxunk policy of the xxmaj government of xxmaj hong xxmaj kong , they xxunk only a xxunk of the industrial buildings of the territory : there were about xxunk industrial buildings in xxmaj hong xxmaj kong in 2003 . xxbos xxmaj good xxunk . xxbos"
1,"xxmaj xxunk xxmaj xxunk were xxunk with a special xxunk xxunk , with xxmaj xxunk xxmaj xxunk 's xxunk of xxmaj xxunk moving xxunk on the xxunk xxunk . xxbos xxmaj she also xxunk with xxmaj xxunk xxmaj xxunk at the xxmaj xxunk xxmaj music xxmaj festival and xxmaj school and privately in xxmaj new xxmaj york xxmaj city . xxbos xxmaj so anyway , xxmaj george , do you know what i was thinking about ? xxbos xxmaj and","xxunk xxmaj xxunk were xxunk with a special xxunk xxunk , with xxmaj xxunk xxmaj xxunk 's xxunk of xxmaj xxunk moving xxunk on the xxunk xxunk . xxbos xxmaj she also xxunk with xxmaj xxunk xxmaj xxunk at the xxmaj xxunk xxmaj music xxmaj festival and xxmaj school and privately in xxmaj new xxmaj york xxmaj city . xxbos xxmaj so anyway , xxmaj george , do you know what i was thinking about ? xxbos xxmaj and you"


Fine-tuning the language model:

In [108]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [109]:
learn.fit_one_cycle(1, 2e-2)
learn.save_encoder('finetuned')

/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.634230,3.457635,0.342403,31.741831,01:23


In [110]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=72)

In [112]:
dls_clas.show_batch(max_n=3)

/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The ser

,text,category
0,"xxbos xxmaj later xxunk were set in a xxunk at xxmaj xxunk xxmaj xxunk xxmaj xxunk xxmaj school , in which xxmaj xxunk appeared to xxunk in for an xxunk xxunk while xxunk xxunk about up and down , high and xxunk , xxunk and xxunk , here and there , good and bad xxunk , xxunk , xxmaj xxunk , xxunk xxunk , trying and trying again , xxunk other people 's xxunk alone , and street - xxunk xxunk .",formal
1,"xxbos xxmaj the album features xxunk of the band 's xxunk singles , xxunk "" how xxmaj do xxmaj ya xxmaj feel "" , and the international - only release "" do n't xxmaj xxunk xxmaj it xxmaj baby "" , as well as two popular xxmaj b - sides , two brand new xxunk , and the original track , "" set xxmaj me xxmaj free "" , which was recorded during the xxmaj xxunk album xxunk .",formal
2,"xxbos xxmaj the xxunk xxunk : xxunk xxunk xxunk xxunk - xxunk or xxunk - xxunk xxunk xxunk head xxunk xxunk of injury is an xxunk xxunk xxunk injury to the xxunk on one side ( xxunk injury ) and the xxunk on the same side ( xxunk or xxunk ) , following which the child is xxunk , xxunk the head on the ground or another xxunk and xxunk injury to the xxunk side of the head .",formal


In [113]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [114]:
learn = learn.load_encoder('finetuned')

In [115]:
learn.fit_one_cycle(1, 2e-2)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,time
0,0.652426,0.575463,0.825000,00:55


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

epoch,train_loss,valid_loss,accuracy,time
0,0.564281,0.335478,1.000000,01:02


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

epoch,train_loss,valid_loss,accuracy,time
0,0.526286,0.219697,0.995000,01:49


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

epoch,train_loss,valid_loss,accuracy,time
0,0.499122,0.130715,0.995000,02:26
1,0.488805,0.117152,0.995000,02:15


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

In [136]:
learn.predict("Rather than always considering policies through the lens of self-interest, it is preferable to do so from an altruistic standpoint.")

/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


('formal', tensor(0), tensor([0.8057, 0.1943]))

In [137]:
save_pickle("formality.pkl", learn)